In [47]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
import re
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import os
# prints current conda environment
print(os.environ['CONDA_DEFAULT_ENV'])

base


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ajay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
df = pd.read_csv("data/question-pairs-dataset/questions.csv")
df = df.dropna(axis=0,how="any")
# df =df.loc[0:350000,:]
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [49]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [50]:
print(df.is_duplicate.value_counts())
255045/len(df)

0    255042
1    149306
Name: is_duplicate, dtype: int64


0.6307561803199224

In [51]:
# taking look at some questions
print("Not duplicate")
print(df.question1[0])
print(df.question2[0])
print()
print("Not duplicate")
print(df.question1[1])
print(df.question2[1])
print()
print(df.question1[5])
print(df.question2[5])

Not duplicate
What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?

Not duplicate
What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?

Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?


In [52]:
def review_to_wordlist(review,remove_stopwords=True):
    # convert sentence to lower case and split it in separate words vector
    words = review.lower().split()
    if remove_stopwords:
        # Optionally remove stop words
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    review_text = " ".join(words)
    # clean the text
    # review_text = "AhbhbhBBjnj2121!??\\'ve@3AB\\'sggFF'll"
    review_text = re.sub("[^A-Za-z0-9(),!.?\'\']"," ",review_text)
    review_text = re.sub("\'s"," 's ",review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    words = review_text.split()
    # shorten words to their stems
    stemmer = SnowballStemmer("english")
    stemmed_words = [stemmer.stem(word) for word in words]
    review_text = " ".join(stemmed_words)
    return(review_text)


In [53]:
def process_questions(question_list, questions,question_list_name):
    for question in questions:
        question_list.append(review_to_wordlist(question))
        if len(question_list) % 10000 == 0:
            progress=(len(question_list)/len(df))*100
            print("{} is {} % complete".format(question_list_name,progress))


In [54]:


questions1 = []
process_questions(question_list=questions1,
                 questions=df.question1,
                 question_list_name="question 1")
print()
questions2 = []
process_questions(question_list=questions2,
                 questions=df.question2,
                 question_list_name="question 2")

question 1 is 2.4731172158635633 % complete
question 1 is 4.9462344317271265 % complete
question 1 is 7.41935164759069 % complete
question 1 is 9.892468863454253 % complete
question 1 is 12.365586079317815 % complete
question 1 is 14.83870329518138 % complete
question 1 is 17.311820511044942 % complete
question 1 is 19.784937726908506 % complete
question 1 is 22.25805494277207 % complete
question 1 is 24.73117215863563 % complete
question 1 is 27.204289374499197 % complete
question 1 is 29.67740659036276 % complete
question 1 is 32.150523806226325 % complete
question 1 is 34.623641022089885 % complete
question 1 is 37.096758237953445 % complete
question 1 is 39.56987545381701 % complete
question 1 is 42.04299266968057 % complete
question 1 is 44.51610988554414 % complete
question 1 is 46.9892271014077 % complete
question 1 is 49.46234431727126 % complete
question 1 is 51.93546153313482 % complete
question 1 is 54.408578748998394 % complete
question 1 is 56.881695964861954 % complete
qu

In [55]:
# Take a look at first 5 pairs of questions
for i in range(5):
    print(questions1[i])
    print(questions2[i])
    print()

step step guid invest share market india
step step guid invest share market

stori kohinoor ( koh i noor ) diamond
would happen indian govern stole kohinoor ( koh i noor ) diamond back

increas speed internet connect use vpn
internet speed increas hack dns

mental lone solv it
find remaind math 23 24 math divid 24 23

one dissolv water quik sugar salt methan carbon di oxid
fish would surviv salt water



In [56]:
# stores the indices of unusable question
invalid_questions = []
for i in range(len(questions1)):
    if not re.search('[aeiouyAEIOUY]',questions1[i]) or not re.search("[aeiouyAEIOUY]",questions2[i]):
        print(i-len(invalid_questions))
        invalid_questions.append(i-len(invalid_questions))
print(len(invalid_questions))

812
1509
3303
3397
3658
3757
6870
7361
7812
9737
10098
10573
10602
11427
12799
13001
13781
15223
15478
17469
17495
18122
18521
19854
20051
20390
20494
20770
22213
23859
26770
28241
28949
31212
31725
31791
32671
32839
33188
33775
34722
35607
36274
36731
37155
38442
40595
43497
44957
47015
51934
52839
53701
53707
53912
54070
55462
55768
57688
60135
65317
67887
67900
71325
75836
76072
78528
81505
82677
88188
88360
89705
90435
91531
93031
96666
99294
99882
103978
104038
104899
105290
106280
106698
107443
108189
108907
108937
109071
109237
113530
115271
121565
122540
124744
124969
127210
127299
128200
129787
130254
130670
133045
134318
135135
136315
136964
138537
138849
138898
139224
141624
143804
144250
144796
146830
147187
148255
150125
150563
150964
151821
153609
156168
156409
157583
158673
159283
159801
161041
163631
165190
165486
165771
166001
168043
168423
169176
169257
170537
172255
174087
174190
174592
174742
174841
175162
176846
178181
179020
181572
182491
184196
184233
184342
1852

In [57]:
# remove invalid questions
for index in invalid_questions:
    df = df[df.id != index]
    questions1.pop(index)
    questions2.pop(index)

In [58]:
# These questions are also unusable, but were not detected initially.
# They were found when the function 'cosine_sim' stopped due to an error

unexpected_invalid_questions = [36459,42272,65936,89323,304866,306827,353917,304865,306826,353916]
unexpected_invalid_questions = np.sort(unexpected_invalid_questions)
for index in unexpected_invalid_questions:
    df = df[df.id != index]
    questions1.pop(index)
    questions2.pop(index)


In [59]:
# use tfidfVectorizer() to transform the questions into vectors
# then compute their cosine similarity
vectorizer = TfidfVectorizer()
def cosine_sim(text1,text2):
    tfidf = vectorizer.fit_transform([text1,text2])
    return ((tfidf*tfidf.T).A)[0,1]

In [60]:
tfidf_scores= []
for i in range(len(questions1)):
    try:
        score = cosine_sim(questions1[i],questions2[i])
        tfidf_scores.append(score)
        if i % 10000 == 0:
            progress = (i/len(questions1))*100
            print("Similarity score is {} % complete".format(round(progress,2)))
    except:
        if i>len(questions1):
            break
        else:
            print(i)
            questions1.pop(i)
            questions2.pop(i)
            df=df[df.id != i]
            print("index is {}".format(i))
        

Similarity score is 0.0 % complete
Similarity score is 2.48 % complete
Similarity score is 4.95 % complete
Similarity score is 7.43 % complete
36459
index is 36459
Similarity score is 9.9 % complete
42272
index is 42272
Similarity score is 12.38 % complete
Similarity score is 14.85 % complete
65936
index is 65936
Similarity score is 17.33 % complete
Similarity score is 19.8 % complete
Similarity score is 22.28 % complete
Similarity score is 24.75 % complete
Similarity score is 27.23 % complete
Similarity score is 29.7 % complete
Similarity score is 32.18 % complete
Similarity score is 34.65 % complete
Similarity score is 37.13 % complete
Similarity score is 39.6 % complete
Similarity score is 42.08 % complete
Similarity score is 44.56 % complete
Similarity score is 47.03 % complete
Similarity score is 49.51 % complete
Similarity score is 51.98 % complete
Similarity score is 54.46 % complete
Similarity score is 56.93 % complete
Similarity score is 59.41 % complete
Similarity score is 61

IndexError: pop index out of range

In [ ]:
# plot the scores
plt.figure(figsize=(12,4))
plt.hist(tfidf_scores,bins=200)
# plt.xlim(0,1)
plt.show()

In [ ]:
# function to report the quality of model
def performance_report(value,score_list):
    scoresVector = []
    for score in score_list:
        if score >=value:
            scoresVector.append(1)
        else:
            scoresVector.append(0)
    accuracyValue = accuracy_score(df.is_duplicate,scoresVector)*100
    print("Accuracy is {}\n".format(accuracyValue))
    print("Confusion Matrix is {}\n".format(confusion_matrix(df.is_duplicate,scoresVector)))
    print("Classification Report is {}\n".format(classification_report(df.is_duplicate,scoresVector)))

In [ ]:
performance_report(value=0.51,score_list=tfidf_scores)

In [ ]:
len(tfidf_scores)

In [ ]:
tfidf_scores[403983]

In [ ]:
tfidf_scores.pop(403983)

In [ ]:
len(df.is_duplicate)